# Generate Test, Evaluation data

In [3]:
import sys
sys.path.append("../")
from src.service.provider import ProviderService

provider = ProviderService()

/home/h4438/miniconda3/envs/uni/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
import os
import pandas as pd
from langchain.text_splitter import RecursiveCharacterTextSplitter

def create_splitter(chunk_size: int = 460, overlap: int = 20) -> RecursiveCharacterTextSplitter:
    """
        get an instance of a text splitter
    """
    word_len_func = lambda e: len(e.split(" "))

    text_splitter = RecursiveCharacterTextSplitter(
        separators=["\n\n\n","\n\n", "\n"],
        chunk_size=chunk_size, chunk_overlap=overlap, 
        length_function=word_len_func, is_separator_regex=False
    )
    return text_splitter

class GeneratePipeline:

    def __init__(self, data_folder: str, ai, size) -> None:
        self.questions = []
        self.records = []
        self.data_folder = data_folder
        self.ai = ai
        self.text_splitter = create_splitter(chunk_size=size, overlap=0)
        return

    def try_split(self):
        files = os.listdir(self.data_folder)
        docs = []
        for f in files:
            if not f.endswith(".md"):
                continue
            DATA = ''
            path = os.path.join(self.data_folder, f)
            with open(path, "r") as f:
                DATA = "\n".join(f.readlines())
            d = self.text_splitter.create_documents([DATA])
            docs = [*docs, *d]
        print(f"{len(files)} -> {len(docs)}")
        return docs

    def run_folder(self):
        files = os.listdir(self.data_folder)
        for f in files:
            if not f.endswith(".md"):
                continue
            self.run_one_file(f)
        return

    def run_one_file(self, file_name:str):
        DATA = ""
        records = []
        questions = []
        path = os.path.join(self.data_folder, file_name)
        with open(path, "r") as f:
            DATA = "\n".join(f.readlines())
        docs = self.text_splitter.create_documents([DATA])
        for doc in docs:
            try:
                record = generate(DATA=doc, questions=questions, ai=self.ai)
            except:
                continue
            record['doc_id'] = file_name.replace(".md", "")
            records.append(record)
            questions.append(record['question'])
        save_file = file_name.replace("md", "csv")
        save_path = os.path.join(self.data_folder, f"test/{save_file}")
        df = pd.DataFrame(records)
        df.to_csv(save_path)
        return

def generate(DATA, questions, ai):
    question_str = ""
    for q in questions:
        question_str += f"- {q}\n"
    PROMPT = f"""Give you this document. Your task is to create multi-choices QA and the question should be a new one
    ```md
    {DATA}
    ```
    Here is a list of your previous asked questions.
    Do not repeat any question from the below list.
    ```
    {question_str}
    ```

    Please generate a new data in Vietnamese as followed (please strictly follow the format below and do not use any "*")
    ```output
    Question: a quesiton about a fact found in the given document, don't repeat yourself or asking similar question from the previously asked question list
    Answer: correct answer for the question. The answer should only be within a line.
    Context: show me where is the answer located in the given document
    ```
    """
    resp = ai(PROMPT)
    print(resp)
    data = parse_data(resp)
    try:
        found = DATA.index(data['context'])
        data['faith'] = found
    except:
        data['faith'] = -1
    return data

def parse_data(data):
    res = {}
    cols = ["Question:", "Answer:", "Context:"]
    for i in range(len(cols)):
        if i < len(cols) - 1:
            s = data.index(cols[i])
            e = data.index(cols[i+1])
            value = data[s:e]
        else:
            value = data[e:]
        key = cols[i].lower().replace(":","").strip()
        res[key] = value.replace(cols[i], "").strip()
    return res

In [5]:
with open("../data/training_program/15277b95-6686-48a8-bc4e-89ef50946af9.md", "r") as f:
    DATA = "\n".join(f.readlines())


ai = provider.get_simple_gemini_pro()
pipeline = GeneratePipeline(data_folder="../data/uni/", ai=ai, size=460)

a = pipeline.try_split()

13 -> 21


In [6]:
pipeline.run_folder()

/home/h4438/miniconda3/envs/uni/lib/python3.9/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.7 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


Question: Năm 2023, TDTU được FIBAA cấp con dấu chất lượng cho bao nhiêu chương trình đào tạo?
Answer: 07 chương trình đào tạo
Context: Năm 2023, TDTU một lần nữa khẳng định chất lượng đào tạo khi được FIBAA cấp con dấu chất lượng cho 07 chương trình đào tạo khác, bao gồm:
Question: Tổ chức nào đã công nhận 4 chương trình đào tạo bậc đại học của TDTU vào tháng 3/2021?
Answer: AUN-QA
Context: Tháng 3/2021, AUN-QA công nhận thêm 4 chương trình đào tạo bậc đại học của TDTU gồm:
Question: Tổ chức nào đã công nhận 3 chương trình đào tạo bậc đại học của TDTU vào tháng 11/2021?
Answer: AUN-QA
Context: Tháng 11/2021, AUN-QA tiếp tục công nhận 3 chương trình đào tạo bậc đại học gồm:
Question: Trường Đại học Tôn Đức Thắng được chứng nhận ISO 9001: 2015 vào năm nào?
Answer: 2008
Context: Trường Đại học Tôn Đức Thắng áp dụng hệ thống quản lý chất lượng theo ISO và được chứng nhận đáp ứng tiêu chuẩn ISO 9001 từ năm 2008.
Question: Trường Đại học Tôn Đức Thắng xếp thứ mấy thế giới theo bảng xếp hạng

# Test prompt

In [38]:
question_str = ""
PROMPT = f"""Give you this document. Your task is to create multi-choices QA and the question should a new one
```md
{DATA}
```
Here is a list of your previous asked questions.
You must not ask any question similar to the list below
```
{question_str}
```

Please generate a new data in Vietnamese as followed (please strictly follow the format below)
```output
"Question": a quesiton about a fact found in the given document, don't repeat yourself or asking similar question from the previously asked question list
"Answer": correct answer for the question. The answer should only be within a line.
"Context": show me where is the answer located in the given document
```
"""
resp = ai(PROMPT)
resp

'"Question": Đối tượng nào được tham dự khóa học bổ sung kiến thức?\n"Answer": Người có bằng tốt nghiệp đại học ngành không phải ngành phù hợp với ngành dự tuyển phải học bổ sung kiến thức để đủ điều kiện dự tuyển chương trình đào tạo trình độ thạc sĩ; Người có bằng tốt nghiệp thạc sĩ ngành phải học bổ sung kiến thức để đủ điều kiện dự tuyển chương trình đào tạo trình độ tiến sĩ.\n"Context": **2) ĐỐI TƯỢNG THAM DỰ** a) Người có bằng tốt nghiệp đại học ngành không phải ngành phù hợp với ngành dự tuyển phải học bổ sung kiến thức để đủ điều kiện dự tuyển chương trình đào tạo trình độ thạc sĩ; b) Người có bằng tốt nghiệp thạc sĩ ngành phải học bổ sung kiến thức để đủ điều kiện dự tuyển chương trình đào tạo trình độ tiến sĩ.'